<a href="https://colab.research.google.com/github/vinhqdang/attack_on_graph_link_prediction/blob/main/EdgePriorityDetector_Cora_New.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 112.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 97.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 61.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12

In [ ]:
!pip install torch-geometric torch-scatter torch-sparse torch-cluster -f https://data.pyg.org/whl/torch-2.0.0+cpu.html

Looking in links: https://data.pyg.org/whl/torch-2.0.0+cpu.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.0/494.0 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.9/750.9 kB 25.6 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.utils import to_scipy_sparse_matrix, from_scipy_sparse_matrix
import random

/usr/local/lib/python3.11/dist-packages/torch_geometric/typing.py:86: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /usr/local/lib/python3.11/dist-packages/torch_scatter/_version_cpu.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/usr/local/lib/python3.11/dist-packages/torch_geometric/typing.py:97: UserWarning: An issue occurred while importing 'torch-cluster'. Disabling its usage. Stacktrace: /usr/local/lib/python3.11/dist-packages/torch_cluster/_version_cpu.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warnings.warn(f"An issue occurred while importing 'torch-cluster'. "
/usr/local/lib/python3.11/dist-packages/torch_geometric/typing.py:124: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /usr/local/lib/python3.11/dist-packages/torch_sparse/_version_cpu.so: undefined symbol: _ZN3c1017RegisterOperatorsD1E

In [ ]:
# Load dataset
dataset = Planetoid(root="/tmp/Cora", name="Cora")
data = dataset[0]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data = data.to(device)

Processing...
Done!


In [ ]:
# Define GCN
class GCN(torch.nn.Module):
    def __init__(self, in_feats, hidden_feats, out_feats):
        super().__init__()
        self.conv1 = GCNConv(in_feats, hidden_feats)
        self.conv2 = GCNConv(hidden_feats, out_feats)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        return self.conv2(x, edge_index)

In [ ]:
# Train the model
model = GCN(data.num_node_features, 16, dataset.num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = F.cross_entropy(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

In [ ]:
# Evaluate before attack
model.eval()
out = model(data.x, data.edge_index)
pred = out.argmax(dim=1)
loss_before = F.cross_entropy(out[data.test_mask], data.y[data.test_mask]).item()
acc_before = (pred[data.test_mask] == data.y[data.test_mask]).sum().item() / data.test_mask.sum().item()


In [ ]:
def epd_pgd_sampling_attack(data, model, budget=20, num_samples=1000):
    A = to_scipy_sparse_matrix(data.edge_index).tolil()
    y = data.y
    N = y.size(0)
    modified = []

    model.eval()
    for _ in range(budget):
        candidates = []
        while len(candidates) < num_samples:
            i = random.randint(0, N - 1)
            j = random.randint(0, N - 1)
            if i != j:
                candidates.append((i, j))

        worst_score = float("inf")  # Invert: we want lowest score
        worst_edge = None

        for (i, j) in candidates:
            A_temp = A.copy()
            A_temp[i, j] = 1 - A_temp[i, j]
            A_temp[j, i] = 1 - A_temp[j, i]
            edge_index_torch, _ = from_scipy_sparse_matrix(A_temp)
            edge_index_torch = edge_index_torch.to(device)

            out = model(data.x, edge_index_torch)
            loss = F.cross_entropy(out[data.train_mask], data.y[data.train_mask])
            score = loss.item()

            if y[i] != y[j] and score < worst_score:
                worst_score = score
                worst_edge = (i, j)

        if worst_edge:
            i, j = worst_edge
            A[i, j] = 1 - A[i, j]
            A[j, i] = A[i, j]
            modified.append((i, j))

    new_edge_index, _ = from_scipy_sparse_matrix(A)
    return new_edge_index.to(device), modified

In [ ]:
# Apply the attack
perturbed_edge_index, modified_links = epd_pgd_sampling_attack(data, model, budget=20, num_samples=1000)


In [ ]:
# Evaluate after attack
model.eval()
out_att = model(data.x, perturbed_edge_index)
pred_att = out_att.argmax(dim=1)
loss_after = F.cross_entropy(out_att[data.test_mask], data.y[data.test_mask]).item()
acc_after = (pred_att[data.test_mask] == data.y[data.test_mask]).sum().item() / data.test_mask.sum().item()


In [ ]:
# Metrics
success = (pred_att[data.test_mask] != data.y[data.test_mask]).sum().item()
total = data.test_mask.sum().item()
asr = success / total
aml = len(modified_links)

In [ ]:
# Print results
print("\nBefore Attack")
print(f"Accuracy: {acc_before:.4f}")
print(f"Loss: {loss_before:.4f}")

print("\nAfter Attack")
print(f"Accuracy: {acc_after:.4f}")
print(f"Loss: {loss_after:.4f}")
print(f"ASR (Attack Success Rate): {asr * 100:.2f}%")
print(f"AML (Avg. Modified Links): {aml}")



Before Attack
Accuracy: 0.8160
Loss: 0.6060

After Attack
Accuracy: 0.8160
Loss: 0.6109
ASR (Attack Success Rate): 18.40%
AML (Avg. Modified Links): 20


In [ ]:
def extract_modified_edges(original_edge_index, perturbed_edge_index):
    # Chuyển sang set để dễ so sánh
    orig_edges = set([tuple(sorted(pair)) for pair in original_edge_index.t().tolist()])
    pert_edges = set([tuple(sorted(pair)) for pair in perturbed_edge_index.t().tolist()])

    # E_aug: Các cạnh được thêm
    E_aug = pert_edges - orig_edges
    # E_atk: Các cạnh bị thay đổi (thêm hoặc xoá)
    E_atk = orig_edges.symmetric_difference(pert_edges)

    return list(E_aug), list(E_atk)

# Gọi hàm
E_aug, E_atk = extract_modified_edges(data.edge_index.cpu(), perturbed_edge_index.cpu())

# Xuất kết quả
print(f"\nTập cạnh đã thêm $E_{{\\text{{aug}}}}$ ({len(E_aug)} cạnh):")
print(E_aug)

print(f"\nTập cạnh bị tấn công $E_{{\\text{{atk}}}}$ ({len(E_atk)} cạnh):")
print(E_atk)



Tập cạnh đã thêm $E_{\text{aug}}$ (20 cạnh):
[(60, 1014), (934, 2295), (7, 1246), (1959, 2427), (61, 1494), (138, 2089), (569, 1079), (162, 1680), (101, 1493), (49, 473), (229, 1761), (69, 2388), (1861, 2418), (76, 2149), (13, 760), (84, 988), (124, 1210), (49, 222), (815, 865), (30, 1700)]

Tập cạnh bị tấn công $E_{\text{atk}}$ (20 cạnh):
[(124, 1210), (60, 1014), (61, 1494), (162, 1680), (84, 988), (934, 2295), (101, 1493), (13, 760), (76, 2149), (815, 865), (569, 1079), (7, 1246), (69, 2388), (49, 473), (229, 1761), (1861, 2418), (1959, 2427), (49, 222), (30, 1700), (138, 2089)]
